In [1]:
import numpy as np
import random
import sys

from chrome_trex import DinoGame

d:\wilfu\Downloads\AI\RL\AlgoritmoGenetico\chrome_trex\dinogame.py:31: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if colorkey is -1:
d:\wilfu\Downloads\AI\RL\AlgoritmoGenetico\chrome_trex\dinogame.py:62: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if colorkey is -1:


In [2]:
# Sinta-se livre para brincar com os valores abaixo

CHANCE_MUT = .20     # Chance de mutação de um peso qualquer
CHANCE_CO = .25      # Chance de crossing over de um peso qualquer
NUM_INDIVIDUOS = 15  # Tamanho da população
NUM_MELHORES = 5     # Número de indivíduos que são mantidos de uma geração para a próxima
N_GERACOES = 100     # Número de gerações

In [3]:
def ordenar_lista(lista, ordenacao, decrescente=True):
    """
    Argumentos da Função:
        lista: lista de números a ser ordenada.
        ordenacao: lista auxiliar de números que define a prioridade da
        ordenação.
        decrescente: variável booleana para definir se a lista `ordenacao`
        deve ser ordenada em ordem crescente ou decrescente.
    Saída:
        Uma lista com o conteúdo de `lista` ordenada com base em `ordenacao`.
    Por exemplo,
        ordenar_lista([2, 4, 5, 6], [7, 2, 5, 4])
        # retorna [2, 5, 6, 4]
        ordenar_lista([1, 5, 4, 3], [3, 8, 2, 1])
        # retorna [5, 1, 4, 3]
    """
    return [x for _, x in sorted(zip(ordenacao, lista), key=lambda p: p[0], reverse=decrescente)]

In [4]:
def populacao_aleatoria(n):
    """
    Argumentos da Função:
        n: Número de indivíduos
    Saída:
        Uma população aleatória. População é uma lista de indivíduos,
        e cada indivíduo é uma matriz 3x10 de pesos (números).
        Os indivíduos podem tomar 3 ações (0, 1, 2) e cada linha da matriz
        contém os pesos associados a uma das ações.
    """

    populacao = []
    for i in range(n):
        populacao.append(np.random.uniform(-10, 10, (3, 10)))
    return populacao

In [5]:
def valor_das_acoes(individuo, estado):
    """
    Argumentos da Função:
        individuo: matriz 3x10 com os pesos do indivíduo.
        estado: lista com 10 números que representam o estado do jogo.
    Saída:
        Uma lista com os valores das ações no estado `estado`. Calcula os valores
        das jogadas como combinações lineares dos valores do estado, ou seja,
        multiplica a matriz de pesos pelo estado.
    """

    return individuo @ estado

In [6]:
def melhor_jogada(individuo, estado):
    """
    Argumentos da Função:
        individuo: matriz 3x10 com os pesos do indivíduo.
        estado: lista com 10 números que representam o estado do jogo.
    Saída:
        A ação de maior valor (0, 1 ou 2) calculada pela função valor_das_acoes.
    """

    return np.argmax(valor_das_acoes(individuo, estado))

In [7]:
def mutacao(individuo):
    """
    Argumentos da Função:
        individuo: matriz 3x10 com os pesos do indivíduo.
    Saída:
        Essa função não tem saída. Ela apenas modifica os pesos do indivíduo,
        de acordo com chance CHANCE_MUT para cada peso.
    """

    for i in range(3):
        for j in range(10):
            if np.random.uniform(0, 1) < CHANCE_MUT:
                individuo[i][j] *= np.random.uniform(-1.5, 1.5)

In [8]:
def crossover(individuo1, individuo2):
    """
    Argumentos da Função:
        individuoX: matriz 3x10 com os pesos do individuoX.
    Saída:
        Um novo indivíduo com pesos que podem vir do `individuo1`
        (com chance 1-CHANCE_CO) ou do `individuo2` (com chance CHANCE_CO),
        ou seja, é um cruzamento entre os dois indivíduos. Você também pode pensar
        que essa função cria uma cópia do `individuo1`, mas com chance CHANCE_CO,
        copia os respectivos pesos do `individuo2`.
    """
    
    filho = individuo1.copy()
    for i in range(3):
        for j in range(10):
            if np.random.uniform(0, 1) < CHANCE_CO:
                filho[i][j] = individuo2[i][j]
    return filho

In [9]:
def calcular_fitness(jogo, individuo):
    """
    Argumentos da Função:
        jogo: objeto que representa o jogo.
        individuo: matriz 3x10 com os pesos do individuo.
    Saída:
        O fitness calculado de um indivíduo. Esse cálculo é feito simulando um
        jogo e calculando o fitness com base nessa simulação. O modo mais simples
        é usando fitness = score do jogo.
    """

    jogo.reset()
    while not jogo.game_over:
        estado = jogo.get_state()
        acao = melhor_jogada(individuo, estado)
        jogo.step(acao)
    return jogo.get_score()

In [10]:
def proxima_geracao(populacao, fitness):
    """
    Argumentos da Função:
        populacao: lista de indivíduos.
        fitness: lista de fitness, uma para cada indivíduo.
    Saída:
        A próxima geração com base na população atual.
        Para criar a próxima geração, segue-se o seguinte algoritmo:
          1. Colocar os melhores indivíduos da geração atual na próxima geração.
          2. Até que a população esteja completa:
             2.1. Escolher aleatoriamente dois indivíduos da geração atual.
             2.2. Criar um novo indivíduo a partir desses dois indivíduos usando
                  crossing over.
             2.3. Mutar esse indivíduo.
             2.4. Adicionar esse indivíduo na próxima geração
    """

    ordenados = ordenar_lista(populacao, fitness)
    proxima_ger = ordenados[:NUM_MELHORES]


    while len(proxima_ger) < NUM_INDIVIDUOS:
        # Selecionar 2 indivíduos, realizar crosover e mutação,
        # e adicionar o novo indivíduo à próxima geração
        ind1, ind2 = random.choices(populacao, k=2)
        filho = crossover(ind1, ind2)
        mutacao(filho)
        proxima_ger.append(filho)

    return proxima_ger

In [11]:
def mostrar_melhor_individuo(jogo, populacao, fitness):
    """
    Argumentos da Função:
        jogo: objeto que representa o jogo.
        populacao: lista de indivíduos.
        fitness: lista de fitness, uma para cada indivíduo.
    Saída:
        Não há saída. Simplesmente mostra o melhor indivíduo de uma população.
    """

    fps_antigo = jogo.fps
    jogo.fps = 100
    ind = populacao[max(range(len(populacao)), key=lambda i: fitness[i])]
    print('Melhor individuo:', ind)
    while True:
        if input('Pressione enter para rodar o melhor agente. Digite q para sair. ') == 'q':
            jogo.fps = fps_antigo
            return
        fit = calcular_fitness(jogo, ind)
        print('Fitness: {:4.1f}'.format(jogo.get_score()))

In [12]:

###############################
# CÓDIGO QUE RODA O ALGORITMO #
###############################



num_geracoes = N_GERACOES
jogo = DinoGame(fps=50_000)

populacao = populacao_aleatoria(NUM_INDIVIDUOS)

print('ger | fitness\n----+-' + '-'*5*NUM_INDIVIDUOS)

for ger in range(num_geracoes):
    fitness = []
    for ind in populacao:
        fitness.append(calcular_fitness(jogo, ind))

    populacao = proxima_geracao(populacao, fitness)

    print('{:3} |'.format(ger),
          ' '.join('{:4d}'.format(s) for s in sorted(fitness, reverse=True)))

    # Opcional: parar se o fitness estiver acima de algum valor (p.ex. 300)
    # if max(fitness) > 300:
    #     break

# Calcula a lista de fitness para a última geração
fitness = []
for ind in populacao:
    fitness.append(calcular_fitness(jogo, ind))


ger | fitness
----+----------------------------------------------------------------------------
  0 |   21   21   21   21   21   21   20   20   19   19   19   19   19   19   19
  1 |   21   21   21   21   21   21   21   21   21   20   20   19   19   19   19
  2 |   21   21   21   21   21   21   21   21   21   21   20   20   20   19   19
  3 |   21   21   21   21   21   21   21   20   20   20   20   20   20   19   19
  4 |   21   21   21   21   21   21   21   21   21   20   20   20   20   20   19
  5 |   21   21   21   21   21   21   21   21   21   21   21   21   20   20   20
  6 |   36   21   21   21   21   21   21   21   21   21   20   20   20   19   19
  7 |   52   22   21   21   21   21   21   21   21   21   20   20   20   19   19
  8 |   53   32   21   21   21   21   21   21   21   21   21   21   20   19   19
  9 |   22   21   21   21   21   21   21   21   20   20   20   20   20   19   19
 10 |   46   21   21   21   21   21   21   20   20   20   20   20   19   19   19
 11 |   45   

In [13]:
mostrar_melhor_individuo(jogo, populacao, fitness)

Melhor individuo: [[ 5.54200928e+00  8.02986583e-02 -9.66146668e-01  1.56471709e-05
   8.83057405e-07  1.81436052e-03 -8.27561863e-03  6.19244717e-01
  -9.11884840e-03  3.26071744e-01]
 [-4.00512032e+00 -2.92939716e-03  4.03830311e+00  2.91431839e-03
  -1.61668552e-02  8.40445654e-02  2.03009195e-01  2.67430205e-01
   3.00598130e-02 -2.53207665e-04]
 [ 1.40212814e-01 -2.21337548e-02  3.23241255e-01 -1.37840975e-02
  -1.30043520e-03 -2.32484758e-02  3.49178637e-01  1.81909485e-01
   7.08558617e-01 -1.28177422e-04]]


: 

: 